In [3]:
import json
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

https://huggingface.co/iamjoon/llama3-8b-finance-analyzer/tree/main

In [2]:
llm = LLM(model="iamjoon/llama3-8b-finance-analyzer")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

INFO 04-08 05:41:29 [config.py:600] This model supports multiple tasks: {'generate', 'reward', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
INFO 04-08 05:41:29 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

INFO 04-08 05:43:04 [weight_utils.py:281] Time spent downloading weights for iamjoon/llama3-8b-finance-analyzer: 89.505068 seconds


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-08 05:43:07 [loader.py:447] Loading weights took 2.77 seconds
INFO 04-08 05:43:07 [gpu_model_runner.py:1273] Model loading took 14.9596 GiB and 92.965476 seconds
INFO 04-08 05:43:13 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/d8f3d150ab/rank_0_0 for vLLM's torch.compile
INFO 04-08 05:43:13 [backends.py:426] Dynamo bytecode transform time: 6.14 s
INFO 04-08 05:43:16 [backends.py:132] Cache the graph of shape None for later use
INFO 04-08 05:43:38 [backends.py:144] Compiling a graph for general shape takes 24.44 s
INFO 04-08 05:43:50 [monitor.py:33] torch.compile takes 30.58 s in total
INFO 04-08 05:43:51 [kv_cache_utils.py:578] GPU KV cache size: 417,952 tokens
INFO 04-08 05:43:51 [kv_cache_utils.py:581] Maximum concurrency for 8,192 tokens per request: 51.02x
INFO 04-08 05:44:18 [gpu_model_runner.py:1608] Graph capturing finished in 27 secs, took 0.51 GiB
INFO 04-08 05:44:18 [core.py:162] init engine (profile, create kv cache, warmup model) to

In [20]:
from vllm import SamplingParams
import json

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["<|eot_id|>"]
)

# 20번부터 30번까지 반복
for i in range(20, 31):
    prompt = input_label_pairs[i]["input"]
    label_text = input_label_pairs[i]["label"]

    print(f"\n\n======= 샘플 {i} =======")

    # LLM 생성
    outputs = llm.generate(prompt, sampling_params)

    # 생성된 텍스트 가져오기
    generated_text = outputs[0].outputs[0].text.strip()

    # JSON 파싱
    try:
        label_json = json.loads(label_text)
        generated_json = json.loads(generated_text)
    except json.JSONDecodeError:
        print("[⚠️ JSON 파싱 실패]")
        print("[Generated Output]\n", generated_text)
        print("\n[정답 Label]\n", label_text)
        continue

    # 비교 키 설정
    if label_json.get("is_stock_related") is True:
        keys_to_check = [
            "is_stock_related",
            "positive_impact_stocks",
            "reason_for_positive_impact",
            "positive_keywords",
            "negative_impact_stocks",
            "reason_for_negative_impact",
            "negative_keywords",
            "summary"
        ]
    else:
        keys_to_check = ["is_stock_related", "summary"]

    # 결과 출력
    print("[Generated Output]")
    for key in keys_to_check:
        print(f"{key}:\n{generated_json.get(key)}\n")

    print('--' * 30)
    
    print("[정답 Label]")
    for key in keys_to_check:
        print(f"{key}:\n{label_json.get(key)}\n")



======= 샘플 20 =======


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.42s/it, est. speed input: 986.48 toks/s, output: 83.32 toks/s]


[Generated Output]
is_stock_related:
False

summary:
한국철도공사 코레일 사장이 나희승으로 임명된 이후 발생한 안전사고로 인해 공공기관 경영평가에서 최하등급을 받았다. 이는 안전사고로 인한 부정적 평가가 반영된 결과로 보인다.

------------------------------------------------------------
[정답 Label]
is_stock_related:
False

summary:
나희승 코레일 사장이 임기 초반에 안전사고와 관련하여 코레일이 공공기관 경영평가에서 최하등급을 받았다는 뉴스입니다. 나 사장의 안전 관리 체계에 대한 책임론이 대두되고 있으며, 이는 그의 정치적 배경과도 관련이 있다는 분석이 제기되고 있는 상황입니다.



======= 샘플 21 =======


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it, est. speed input: 941.44 toks/s, output: 82.39 toks/s]


[Generated Output]
is_stock_related:
False

summary:
이 기사는 10년 전의 주요 뉴스들을 회상하며, 스마트폰의 대중화로 인한 DMB 서비스의 쇠퇴, 치킨집 거리 제한 규제, 과도한 주류 판매로 인한 폭력 문제 등에 대해 다루고 있습니다. 또한 이시영의 복싱 경력과 강남 아파트 가격 상승에 대해 언급하고 있습니다. 주식 시장과 직접적인 연관은 없지만, 소비 문화와 사회적 현상에 대한 통찰을 제공하고 있습니다.

------------------------------------------------------------
[정답 Label]
is_stock_related:
False

summary:
10년전N이라는 연재물에서 2012년의 주요 뉴스를 다루고 있다. 스마트폰의 대중화로 인해 DMB 서비스는 사라졌고, 치킨 가맹점의 거리 제한 규제가 시행되었으며, 술 구매 시 경품 제공에 제재가 있었다. 배우 이시영의 과거 복싱 경력이 소개되었고, 서울 강남 아파트의 가격 양극화 상황이 언급되었다.



======= 샘플 22 =======


...

[Generated Output]
is_stock_related:
False

summary:
SRT 열차가 탈선하여 11명이 부상을 입었고, 사고 원인 파악을 위해 현장에 유관기관 긴급구조대가 투입되었다. 상·하행선은 지연 운행 중이며, 고속열차를 이용하는 고객들에게는 타 교통수단 이용을 권장하고 있다.

------------------------------------------------------------
[정답 Label]
is_stock_related:
False

summary:
부산에서 서울로 향하던 SRT 열차가 대전조차장역 인근에서 탈선하여 11명이 부상당했으며, 상하행선 열차 운행이 지연되고 있다. 사고 원인에 대한 조사가 진행 중이며, 국토교통부와 관련 기관이 현장에 투입되었다.



======= 샘플 29 =======


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.29s/it, est. speed input: 612.34 toks/s, output: 83.36 toks/s]


[Generated Output]
is_stock_related:
True

positive_impact_stocks:
['기아']

reason_for_positive_impact:
기아의 전년 대비 판매량 증가로 인해 긍정적인 영향을 받을 수 있다.

positive_keywords:
['기아', '스포티지', '셀토스', '판매 증가']

negative_impact_stocks:
[]

reason_for_negative_impact:


negative_keywords:
[]

summary:
기아는 지난달 글로벌 시장에서 전년 동월 대비 1.8% 증가한 25만8610대의 판매를 기록했으며, 스포티지가 4만3201대로 가장 많이 판매되었다. 반도체 부품 수급 문제에도 불구하고 기아는 유연한 반도체 배분과 생산 일정 조정으로 공급 지연에 대응하고 있다.

------------------------------------------------------------
[정답 Label]
is_stock_related:
True

positive_impact_stocks:
['기아']

reason_for_positive_impact:
기아의 6월 글로벌 판매 실적이 전년 동월 대비 1.8% 증가하며 기대 이상의 실적을 보였고, 반도체 수급난 속에서도 스포티지 등 주요 차종의 판매 호조가 이어지고 있다는 점에서 긍정적인 영향을 줄 것으로 보인다.

positive_keywords:
['기아', '스포티지', '글로벌 시장', '반도체 수급난']

negative_impact_stocks:
[]

reason_for_negative_impact:


negative_keywords:
[]

summary:
기아의 6월 글로벌 판매 실적이 반도체 수급난에도 불구하고 전년 동월 대비 1.8% 증가한 25만8610대를 기록했다. 스포티지가 가장 많이 팔렸으며, 반도체 문제 개선을 통해 최대 실적을 기록했고, 향후 신차 출시로 시장 점유율과 수익성을 확대할 계획이다.


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it, est. speed input: 594.76 toks/s, output: 83.91 toks/s]

[Generated Output]
is_stock_related:
True

summary:
KOSPI 지수가 2.13% 하락하여 2,292.01 포인트로 마감했다.

------------------------------------------------------------
[정답 Label]
is_stock_related:
False

summary:
코스피 지수가 49.77포인트 하락하여 2292.01포인트로 마감했다는 뉴스이다.

